# Step 1: Install Dependencies

In [ ]:
!pip install fastapi uvicorn pyngrok python-multipart nest_asyncio requests


# Step 2: Configure Ngrok
Paste your token below.

In [ ]:
from pyngrok import ngrok

# PASTE YOUR TOKEN BELOW
NGROK_TOKEN = "YOUR_NGROK_TOKEN_HERE"

ngrok.set_auth_token(NGROK_TOKEN)

# Step 3: Run the Investigator Node

In [ ]:
# === JURIS-CAPE WORKER NODE B: THE INVESTIGATOR ===

import uvicorn
import asyncio
import os
import time
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse
from pyngrok import ngrok
from pydantic import BaseModel

# --- CONFIGURATION ---
SWARM_SECRET = "change-me-in-prod-secure-swarm-key"
ROLE = "investigator"

app = FastAPI()

@app.middleware("http")
async def verify_secret(request: Request, call_next):
    if request.url.path in ["/docs", "/openapi.json"]:
        return await call_next(request)
    secret = request.headers.get("X-Swarm-Secret")
    if secret != SWARM_SECRET:
        return JSONResponse(status_code=403, content={"detail": "Unauthorized"})
    return await call_next(request)

class MediaRequest(BaseModel):
    file_url: str
    type: str # 'audio' or 'video'

@app.post("/analyze_media")
async def analyze_media(payload: MediaRequest):
    print(f"Analyzing {payload.type}: {payload.file_url}")
    return {"transcript": "[MOCK] Transcribed text...", "visual_summary": "[MOCK] Video description..."}

# --- RUNNER ---
print("Cleaning up old tunnels...")
ngrok.kill()
os.system("pkill ngrok")
time.sleep(1)

try:
    ngrok_tunnel = ngrok.connect(8000)
    print("\n=== PUBLIC URL ===")
    print(ngrok_tunnel.public_url)
    print("====================\n")

    config = uvicorn.Config(app, port=8000)
    server = uvicorn.Server(config)
    await server.serve()
except Exception as e:
    print(f"FATAL ERROR: {e}")
